### Segmentation evaluation for training and testing set of baseline
**F1 score from time domain**

In [1]:
from shapely.ops import cascaded_union
import matplotlib.pyplot as plt
import geopandas as gpd
import multiprocessing
import pandas as pd
import numpy as np
import skimage.io
import tqdm
import glob
import math
import gdal
import time
import sys
import os

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 16})
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300
import matplotlib
# matplotlib.use('Agg') # non-interactive
from sklearn.metrics import roc_auc_score, average_precision_score, roc_curve, auc

import solaris as sol
from solaris.utils.core import _check_gdf_load
from solaris.raster.image import create_multiband_geotiff 

# import from data_postproc_funcs
module_path = os.path.abspath(os.path.join('../src/'))
if module_path not in sys.path:
    sys.path.append(module_path)
from sn7_baseline_postproc_funcs import map_wrapper, multithread_polys, \
        calculate_iou, track_footprint_identifiers, \
        sn7_convert_geojsons_to_csv

/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Midgard/home/hfang/miniconda3/envs/solaris/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Midgard/home/hfang

In [2]:
def dice_coef(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    return (2. * np.sum(intersection)) / (np.sum(y_true) + np.sum(y_pred))

def iou_coef(y_true, y_pred):
    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou = np.sum(intersection) / np.sum(union)
    return iou

In [ ]:
def group(raw_name, grouped_name):
    im_list = sorted([z for z in os.listdir(os.path.join(raw_name)) if z.endswith('.npy')])
    df = pd.DataFrame({'image': im_list})
    roots = [z.split('mosaic_')[-1].split('.npy')[0] for z in df['image'].values]
    df['root'] = roots
    # copy files
    for idx, row in df.iterrows():
        in_path_tmp = os.path.join(raw_name, row['image'])
        out_dir_tmp = os.path.join(grouped_name, row['root'], 'masks')
        os.makedirs(out_dir_tmp, exist_ok=True)
        cmd = 'cp ' + in_path_tmp + ' ' + out_dir_tmp
#         print("cmd:", cmd)
        os.system(cmd)   

In [ ]:
raw_name = '/Midgard/home/hfang/temporal_CD/Berzelius_HRNet_SN7/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_alpha_1_train/npy_compose'
grouped_name = '/Midgard/home/hfang/temporal_CD/Berzelius_HRNet_SN7/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_alpha_1_train/grouped'
group(raw_name, grouped_name)

In [3]:
def dice_iou_time_domain(pred_top_dir, im_top_dir, aoi, left, right, outfile, mode):
    # outfile: save file to .csv
    # mode: baseline 1024*1024, winner 3072*3072
    # aoi: given aoi
    # left: left margin of theta
    # right: right margin of theta
    
    df = pd.DataFrame()
    
#     aois = sorted([f for f in os.listdir(os.path.join(im_top_dir))
#                    if os.path.isdir(os.path.join(im_top_dir, f))
#                   and f != 'list'])

    # save aoi and the bound of theta to a dictionary
    
    print(i, "aoi:", aoi)
    
    for theta in range(21):
        per_theta = (right - left) / 20
        threshold = left + theta * per_theta
        
        print(threshold)

        pred_dir = os.path.join(pred_top_dir, 'grouped', aoi, 'masks')
        im_dir = os.path.join(im_top_dir, aoi, 'images_masked')
        gt_dir = os.path.join(im_top_dir, aoi, 'masks')

        im_list = sorted([z for z in os.listdir(im_dir) if z.endswith('.tif')])

        for j, f in enumerate(im_list):        
            sample_mask_name = f
            if mode == 'baseline':
                sample_mask_path = os.path.join(pred_dir, sample_mask_name)
            elif mode == 'winner':
                sample_mask_path = os.path.join(pred_dir, sample_mask_name).replace('.tif', '.npy')
            sample_im_path = os.path.join(im_dir, sample_mask_name)
            sample_gt_path = os.path.join(gt_dir, sample_mask_name).replace('.tif', '_Buildings.tif')

            image = skimage.io.imread(sample_im_path)
            if mode == 'baseline':
                mask_image = skimage.io.imread(sample_mask_path)
            elif mode == 'winner':
                mask_image = np.load(sample_mask_path)
            gt_image = skimage.io.imread(sample_gt_path)

            norm = (mask_image - np.min(mask_image)) / (np.max(mask_image) - np.min(mask_image))

            gt_image = gt_image / 255

            if mode == 'winner':
                tmp = np.zeros((1024, 1024))
                tmp[:gt_image.shape[0],:gt_image.shape[1]] = gt_image
                gt_image = np.repeat(tmp, 3, axis=0)
                gt_image = np.repeat(gt_image, 3, axis=1)

            # threshold
            norm = np.where(norm > threshold, 1, 0)

            dice = dice_coef(gt_image, norm)
            iou = iou_coef(gt_image, norm)

            row1 = {'AOI': aoi, 'Dice': dice, 'IOU': iou, 'Frame': j, 'threshold': threshold}
            df = df.append(row1, ignore_index=True)
        
    df.to_csv(outfile, index=False, header=True)

In [4]:
dict_aoi_bound = {
    "L15-0387E-1276N_1549_3087_13": [0.2, 0.4],
    "L15-0566E-1185N_2265_3451_13": [0.2, 0.8],
    "L15-0632E-0892N_2528_4620_13": [0.3, 0.5],
    "L15-1015E-1062N_4061_3941_13": [0.2, 0.4],
    "L15-1200E-0847N_4802_4803_13": [0.2, 0.3],
    "L15-1276E-1107N_5105_3761_13": [0.2, 0.3],
    "L15-1438E-1134N_5753_3655_13": [0.2, 0.3],
    "L15-1615E-1206N_6460_3366_13": [0.1, 0.4],
    "L15-1690E-1211N_6763_3346_13": [0.2, 0.4],
    "L15-1848E-0793N_7394_5018_13": [0.2, 0.4]
}

In [5]:
# Set prediction and image (ground truth) directories (edit appropriately)
pred_top_dir = '/local_storage/users/hfang/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_alpha_1_train'
im_top_dir = '/local_storage/datasets/sn7_winner_split/test_public'

aois = sorted([f for f in os.listdir(os.path.join(im_top_dir))
               if os.path.isdir(os.path.join(im_top_dir, f))
              and f != 'list'])

for i, aoi in enumerate(aois):
    dice_iou_time_domain(pred_top_dir, im_top_dir, aoi, dict_aoi_bound[aoi][0], dict_aoi_bound[aoi][1], 'segmentation_infer_time_domain/segmentation_infer_time_domain_tcr_v1_alpha_1' + aoi + '.csv', mode='winner')

0 aoi: L15-0387E-1276N_1549_3087_13
0.2
0.21000000000000002
0.22
0.23
0.24000000000000002
0.25
0.26
0.27
0.28
0.29000000000000004
0.30000000000000004
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
1 aoi: L15-0566E-1185N_2265_3451_13
0.2
0.23
0.26
0.29000000000000004
0.32000000000000006
0.35000000000000003
0.38000000000000006
0.41000000000000003
0.44000000000000006
0.4700000000000001
0.5
0.53
0.56
0.5900000000000001
0.6200000000000001
0.6500000000000001
0.6800000000000002
0.7100000000000002
0.7400000000000002
0.77
0.8
2 aoi: L15-0632E-0892N_2528_4620_13
0.3
0.31
0.32
0.32999999999999996
0.33999999999999997
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.44999999999999996
0.45999999999999996
0.47
0.48
0.49
0.5
3 aoi: L15-1015E-1062N_4061_3941_13
0.2
0.21000000000000002
0.22
0.23
0.24000000000000002
0.25
0.26
0.27
0.28
0.29000000000000004
0.30000000000000004
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
4 aoi: L15-1200E-0847N_4802_4803_13
0.2
0.20500000000000002
0.2100000000000000

In [6]:
# Set prediction and image (ground truth) directories (edit appropriately)
pred_top_dir = '/local_storage/users/hfang/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_alpha_05_train'
im_top_dir = '/local_storage/datasets/sn7_winner_split/test_public'

aois = sorted([f for f in os.listdir(os.path.join(im_top_dir))
               if os.path.isdir(os.path.join(im_top_dir, f))
              and f != 'list'])

for i, aoi in enumerate(aois):
    dice_iou_time_domain(pred_top_dir, im_top_dir, aoi, dict_aoi_bound[aoi][0], dict_aoi_bound[aoi][1], 'segmentation_infer_time_domain/segmentation_infer_time_domain_tcr_v1_alpha_05'  + aoi + '.csv', mode='winner')

0 aoi: L15-0387E-1276N_1549_3087_13
0.2
0.21000000000000002
0.22
0.23
0.24000000000000002
0.25
0.26
0.27
0.28
0.29000000000000004
0.30000000000000004
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
1 aoi: L15-0566E-1185N_2265_3451_13
0.2
0.23
0.26
0.29000000000000004
0.32000000000000006
0.35000000000000003
0.38000000000000006
0.41000000000000003
0.44000000000000006
0.4700000000000001
0.5
0.53
0.56
0.5900000000000001
0.6200000000000001
0.6500000000000001
0.6800000000000002
0.7100000000000002
0.7400000000000002
0.77
0.8
2 aoi: L15-0632E-0892N_2528_4620_13
0.3
0.31
0.32
0.32999999999999996
0.33999999999999997
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.44999999999999996
0.45999999999999996
0.47
0.48
0.49
0.5
3 aoi: L15-1015E-1062N_4061_3941_13
0.2
0.21000000000000002
0.22
0.23
0.24000000000000002
0.25
0.26
0.27
0.28
0.29000000000000004
0.30000000000000004
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
4 aoi: L15-1200E-0847N_4802_4803_13
0.2
0.20500000000000002
0.2100000000000000

In [7]:
# Set prediction and image (ground truth) directories (edit appropriately)
pred_top_dir = '/local_storage/users/hfang/berzelius_seg_hrnet_tcr_w48_512x512_sgd_lr1e-3_wd1e-4_bs_64_epoch70_train'
im_top_dir = '/local_storage/datasets/sn7_winner_split/test_public'

aois = sorted([f for f in os.listdir(os.path.join(im_top_dir))
               if os.path.isdir(os.path.join(im_top_dir, f))
              and f != 'list'])

for i, aoi in enumerate(aois):
    dice_iou_time_domain(pred_top_dir, im_top_dir, aoi, dict_aoi_bound[aoi][0], dict_aoi_bound[aoi][1], 'segmentation_infer_time_domain/segmentation_infer_time_domain_tcr_v1_alpha_09' + aoi + '.csv', mode='winner')

0 aoi: L15-0387E-1276N_1549_3087_13
0.2
0.21000000000000002
0.22
0.23
0.24000000000000002
0.25
0.26
0.27
0.28
0.29000000000000004
0.30000000000000004
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
1 aoi: L15-0566E-1185N_2265_3451_13
0.2
0.23
0.26
0.29000000000000004
0.32000000000000006
0.35000000000000003
0.38000000000000006
0.41000000000000003
0.44000000000000006
0.4700000000000001
0.5
0.53
0.56
0.5900000000000001
0.6200000000000001
0.6500000000000001
0.6800000000000002
0.7100000000000002
0.7400000000000002
0.77
0.8
2 aoi: L15-0632E-0892N_2528_4620_13
0.3
0.31
0.32
0.32999999999999996
0.33999999999999997
0.35
0.36
0.37
0.38
0.39
0.4
0.41
0.42
0.43
0.44
0.44999999999999996
0.45999999999999996
0.47
0.48
0.49
0.5
3 aoi: L15-1015E-1062N_4061_3941_13
0.2
0.21000000000000002
0.22
0.23
0.24000000000000002
0.25
0.26
0.27
0.28
0.29000000000000004
0.30000000000000004
0.31
0.32
0.33
0.34
0.35
0.36
0.37
0.38
0.39
0.4
4 aoi: L15-1200E-0847N_4802_4803_13
0.2
0.20500000000000002
0.2100000000000000